In [2]:
# Run cell before starting workspace - loads necessary packages
library(tidyverse)
library(repr)
library(readxl)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [11]:
# Read data into df
url <- "https://raw.githubusercontent.com/incribo-inc/stroke_prediction/main/stroke_prediction_dataset.csv"
stroke <- read.csv(url)


In [13]:
# get list of column names
colnames(data)

# wrangling
stroke_selected <- stroke |>
    select(Age, Gender, Hypertension, Heart.Disease, Alcohol.Intake, 
           Physical.Activity, Stroke.History, Family.History.of.Stroke,
           Dietary.Habits, Blood.Pressure.Levels, Diagnosis)
stroke_selected

[1] "Patient.ID"               "Patient.Name"            
 [3] "Age"                      "Gender"                  
 [5] "Hypertension"             "Heart.Disease"           
 [7] "Marital.Status"           "Work.Type"               
 [9] "Residence.Type"           "Average.Glucose.Level"   
[11] "Body.Mass.Index..BMI."    "Smoking.Status"          
[13] "Alcohol.Intake"           "Physical.Activity"       
[15] "Stroke.History"           "Family.History.of.Stroke"
[17] "Dietary.Habits"           "Stress.Levels"           
[19] "Blood.Pressure.Levels"    "Cholesterol.Levels"      
[21] "Symptoms"                 "Diagnosis"

Age,Gender,Hypertension,Heart.Disease,Alcohol.Intake,Physical.Activity,Stroke.History,Family.History.of.Stroke,Dietary.Habits,Blood.Pressure.Levels,Diagnosis
<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>
56,Male,0,1,Social Drinker,Moderate,0,Yes,Vegan,140/108,Stroke
80,Male,0,0,Never,Low,0,No,Paleo,146/91,Stroke
26,Male,1,1,Rarely,High,0,Yes,Paleo,154/97,Stroke
73,Male,0,0,Frequent Drinker,Moderate,0,No,Paleo,174/81,No Stroke
51,Male,1,1,Rarely,Low,0,Yes,Pescatarian,121/95,Stroke
62,Female,0,0,Social Drinker,High,0,No,Gluten-Free,132/64,Stroke
40,Female,1,0,Never,Low,1,No,Vegetarian,178/105,No Stroke
61,Female,0,1,Rarely,Moderate,1,No,Vegan,179/72,Stroke
72,Female,1,1,Rarely,High,1,No,Vegetarian,141/106,Stroke
